## Backtesting
### Implementación de una cartera histórica
En este cuaderno mostramos como implementar de forma eficiente una cartera de inversión
que registra diariamente sus posiciones y la valoración. Las ideas centrales son:
- Simular con posiciones y valoración es una alternativa más realista frente a simplemente componer los retornos de una estrategia
    - Se pueden incorporar costes que no tengan un esquema proporcinal para todos los activos
    - Se pueden considerar restricciones de volumen en la negociación
    - Se pueden implementar ajustes a la cartera que dependen de estados previos
- Cuando no hay operativa en cada punto de tiempo, trabajar con las diferencias en las posiciones/caja nos permite mantener algunas ventajas de los cálculos eficientes que sean vectorizados 

____

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import pickle

### Datos 
Asumiremos por un tema se simplicidad en los ejemplos, que nuestro universo de acciones invertibles son las siguientes acciones del IBEX35.  

In [ ]:
with open('../data/benchmark.pkl', 'rb') as handle:
    benchmarks = pickle.load(handle)
benchmark = benchmarks['ibex_div'].close

In [ ]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

Construimos un dataframe con los precios de cierre

In [ ]:
close_series = {ticker: df.close
                for ticker, df in stock_data.items()}
stock_df = pd.DataFrame(close_series)
stock_df = stock_df.loc['2017':]
stock_df = stock_df.dropna(how='all', axis=1)

In [ ]:
stock_df.head()

___

Registro de cambios de posición
 - las compras van como valores positivos
 - las ventas van como valores negativos   

In [ ]:
delta_trades = pd.DataFrame(
    dtype=np.float,
    index=stock_df.index,
    columns=stock_df.columns
)
delta_trades.head()

____
### Estrategia con Medias Móviles


Hacemos una función que calcule las señales de entrada y salida

In [ ]:
def movaverage_states(vseries, win):
    sma = vseries.rolling(win).mean()
    signal_states = (vseries > sma).astype(float)
    trading_states = signal_states.shift(1)
    trading_states.iloc[0] = 0
    return trading_states

In [ ]:
def sma_signals(vseries, win):
    states = movaverage_states(vseries, win)
    signals = states.diff().dropna()
    return signals

In [ ]:
win = 50
iseries = stock_df['SAN']
sma_states = movaverage_states(iseries, win)
signals = sma_states.diff().dropna()

In [ ]:
signals[signals != 0]

Cada día podemos saber las compras y ventas

In [ ]:
all_signals = stock_df.apply(sma_signals, win=win)
all_signals

In [ ]:
row = all_signals.loc['2017-05-11']
row[row != 0]

In [ ]:
all_signals.any(axis=1)

_____
Verificamos que haya precio en cada señal

In [ ]:
stock_df['TRE'].isna().any()

In [ ]:
def check_prices(signals, prices):
    trading = signals[signals != 0]
    trade_prices = prices.loc[trading.index]
    return trade_prices.isna().any()

In [ ]:
checks = {ticker: check_prices(all_signals[ticker], stock_df[ticker])
          for ticker in stock_df.columns}
pd.Series(checks)

____
### Cartera Histórica

Elementos a considerar:
 - Un registro de cambios de posición (*delta_trades*)
 - Un registro de cambios de efectivo (*delta_cash*)
 - Las posiciones y la caja son la suma acumulada de los cambios
 - La valoraciones son las posiciones por el precio del día
 - Las inversiones son la suma de las valoraciones
 - El patrimonio de la estrategia son las inversiones más la caja
 
Opcionalmente se podrían incluir los costes explícitos en el proceso iterativo de la cartera

In [ ]:
init_capital = 100_000

In [ ]:
delta_cash = pd.Series(0, index=stock_df.index)
delta_cash.iloc[0] = init_capital
delta_shares = pd.DataFrame(0, index=stock_df.index, columns=stock_df.columns)

for idate, isignals in all_signals.iterrows():    
    daybuys = isignals[isignals == 1]
    daysells = isignals[isignals == -1]
    
    if daybuys.count() == 0 and daysells.count() == 0:
        continue

    # actualizar portfolio 
    cash = delta_cash.cumsum()
    posiciones = delta_shares.cumsum()
    valoracion = posiciones * stock_df
    inversiones = valoracion.sum(axis=1)
    equity = inversiones + cash
    
    day_cash = cash.loc[idate]
    # limite de asignación a un 10% de la cartera
    alloc_limit = equity.loc[idate] * 0.1

    in_money, out_money = 0, 0
    # compras
    if daybuys.count() > 0:
        tk_money = min(day_cash/daybuys.count(), alloc_limit)
        buy_shares = np.floor(tk_money/stock_df.loc[idate, daybuys.index])
        out_money = (buy_shares * stock_df.loc[idate, daybuys.index]).sum()
        delta_shares.loc[idate, daybuys.index] = buy_shares

    # ventas
    if daysells.count() > 0:
        sell_shares = posiciones.loc[idate, daysells.index]
        in_money = (sell_shares * stock_df.loc[idate, daysells.index]).sum()
        delta_shares.loc[idate, daysells.index] = -sell_shares

    delta_cash.loc[idate] = in_money - out_money

In [ ]:
# ultima actualizacion
cash = delta_cash.cumsum()
posiciones = delta_shares.cumsum()
valoracion = posiciones * stock_df
inversiones = valoracion.sum(axis=1)
equity = inversiones + cash

In [ ]:
show_df = pd.DataFrame({
    'inversiones': inversiones,
    'efectivo': cash,
    'patrimonio': equity
})
show_df.plot(figsize=(10,6))

In [ ]:
show_df.tail()

In [ ]:
bm = benchmark.reindex(stock_df.index)
pasive_invest = init_capital*(bm/bm.iloc[0])

In [ ]:
compare_df = pd.DataFrame({
    'SMA': equity,
    'benchmark': pasive_invest
})

compare_df.plot(figsize=(10,6))

**OJO**: Las estrategias de este estilo necesitan una ventana para construir sus primeras señales. A discreción tenemos que decidir.
 - Si la ventana inicial se considera o no para la comparación
 - Si construimos las señales a con precios fuera del punto de referencia para la inversión inicial

____

Verificamos ahora la el porcentaje de asignación de cada acción


In [ ]:
allocations = valoracion.div(equity, axis=0)
figs = allocations.plot(subplots=True, figsize=(10, 70))

____
### Ejercicio Propuesto (Opcional)
- Comparar el benchmark con la alternativa de utilizar los **precios de apertura** como forma
de adelantar la ejecución respecto a las señales de trading
- Implementar un esquema de comisiones que incluya 2euros + 0.02% sobre el efectivo en las operaciones de venta
